# Advanced LangChain Patterns

This notebook covers advanced agent patterns:

1. **Structured Output** - Extracting typed data from LLM responses
2. **Dynamic Prompts** - Runtime prompt customization via middleware
3. **Human-in-the-Loop (HITL)** - Requiring approval before tool execution

---

## Setup

In [ ]:
%pip install -qU langchain langchain-openai langchain-community langgraph

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

---

# Part 1: Structured Output

Often you need structured data from LLMs - not just text. LangChain's `response_format` parameter ensures the output matches a specific schema.

## 1.1 Using TypedDict

In [ ]:
from typing_extensions import TypedDict
from langchain.agents import create_agent

# Define the output schema
class ContactInfo(TypedDict):
    name: str
    email: str
    phone: str

# Create agent with structured output
contact_extractor = create_agent(
    model="openai:gpt-4o-mini", 
    response_format=ContactInfo
)

In [ ]:
# Extract contact info from unstructured text
recorded_conversation = """
We talked with John Doe. He works over at Example Corp. 
His number is five, five, five, one two three, four five six seven. 
And his email was john at example.com. 
He wanted to order 50 boxes of supplies.
"""

result = contact_extractor.invoke(
    {"messages": recorded_conversation}
)

# Access the structured response
print("Structured output:")
print(result["structured_response"])

In [ ]:
# Access individual fields
contact = result["structured_response"]
print(f"Name: {contact['name']}")
print(f"Email: {contact['email']}")
print(f"Phone: {contact['phone']}")

## 1.2 Using Pydantic Models

Pydantic provides validation and richer type support:

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional

class MeetingNote(BaseModel):
    title: str = Field(description="Brief title for the meeting")
    attendees: List[str] = Field(description="List of people who attended")
    action_items: List[str] = Field(description="Tasks to be completed")
    next_meeting: Optional[str] = Field(description="Date/time of next meeting if mentioned")
    summary: str = Field(description="2-3 sentence summary of the meeting")

meeting_summarizer = create_agent(
    model="openai:gpt-4o-mini",
    response_format=MeetingNote,
    system_prompt="Extract structured meeting notes from the transcript."
)

In [ ]:
transcript = """
Meeting started at 2pm with Alice, Bob, and Charlie present.

Alice: Let's discuss the Q4 roadmap. We need to finalize features by Friday.

Bob: I can handle the API documentation. Should be done by Wednesday.

Charlie: I'll review the security audit report and send recommendations.

Alice: Great. Let's meet again next Monday at 10am to review progress.

Meeting ended at 2:30pm.
"""

result = meeting_summarizer.invoke({"messages": transcript})
notes = result["structured_response"]

print(f"Title: {notes.title}")
print(f"Attendees: {', '.join(notes.attendees)}")
print(f"\nAction Items:")
for item in notes.action_items:
    print(f"  - {item}")
print(f"\nNext Meeting: {notes.next_meeting}")
print(f"\nSummary: {notes.summary}")

## 1.3 Complex Nested Structures

In [ ]:
from typing import Literal

class Task(BaseModel):
    description: str
    assignee: str
    priority: Literal["high", "medium", "low"]
    estimated_hours: float

class ProjectPlan(BaseModel):
    project_name: str
    objective: str
    tasks: List[Task]
    total_estimated_hours: float
    risks: List[str]

project_planner = create_agent(
    model="openai:gpt-4o-mini",
    response_format=ProjectPlan,
    system_prompt="Create a detailed project plan from the given requirements."
)

In [ ]:
requirements = """
We need to build a customer feedback dashboard.
Team: Sarah (frontend), Mike (backend), Lisa (design)
Must be done in 2 weeks.
Features: sentiment analysis, charts, export to PDF.
"""

result = project_planner.invoke({"messages": requirements})
plan = result["structured_response"]

print(f"Project: {plan.project_name}")
print(f"Objective: {plan.objective}")
print(f"\nTasks:")
for task in plan.tasks:
    print(f"  [{task.priority.upper()}] {task.description}")
    print(f"      Assignee: {task.assignee}, Est: {task.estimated_hours}h")
print(f"\nTotal Hours: {plan.total_estimated_hours}")
print(f"\nRisks:")
for risk in plan.risks:
    print(f"  - {risk}")

---

# Part 2: Dynamic Prompts

Sometimes you need to customize the system prompt at runtime based on context. LangChain middleware enables this with `@dynamic_prompt`.

## 2.1 Role-Based Access Control

In [ ]:
from dataclasses import dataclass
from langchain_community.utilities import SQLDatabase
from langchain_core.tools import tool
from langgraph.runtime import get_runtime
from langchain.agents.middleware.types import ModelRequest, dynamic_prompt

db = SQLDatabase.from_uri("sqlite:///./assets-resources/Chinook.db")

@dataclass
class RuntimeContext:
    is_employee: bool  # Access control flag
    db: SQLDatabase

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite SELECT query and return results."""
    runtime = get_runtime(RuntimeContext)
    try:
        return runtime.context.db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [ ]:
SYSTEM_PROMPT_TEMPLATE = """You are a SQLite analyst.

Rules:
- Use execute_sql for SELECT queries only.
- Limit to 5 rows unless asked otherwise.
{table_limits}
- If errors occur, revise and retry.
"""

@dynamic_prompt
def access_controlled_prompt(request: ModelRequest) -> str:
    """Generate prompt based on user's access level."""
    if not request.runtime.context.is_employee:
        # Non-employees have limited table access
        table_limits = "- You can ONLY access: Album, Artist, Genre, Playlist, PlaylistTrack, Track."
    else:
        # Employees have full access
        table_limits = "- You have access to all tables."
    
    return SYSTEM_PROMPT_TEMPLATE.format(table_limits=table_limits)

In [ ]:
from langchain.agents import create_agent

# Create agent with dynamic prompt middleware
access_controlled_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[execute_sql],
    middleware=[access_controlled_prompt],  # <-- Dynamic prompt
    context_schema=RuntimeContext,
)

In [ ]:
# Non-employee: Should be denied access to customer data
question = "What is the most costly purchase by Frank Harris?"

print("=== Non-Employee Access ===")
for step in access_controlled_agent.stream(
    {"messages": question},
    context=RuntimeContext(is_employee=False, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
# Employee: Should have full access
print("\n=== Employee Access ===")
for step in access_controlled_agent.stream(
    {"messages": question},
    context=RuntimeContext(is_employee=True, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

---

# Part 3: Human-in-the-Loop (HITL)

For sensitive operations, you may want human approval before the agent executes tools. The `HumanInTheLoopMiddleware` provides this capability.

## 3.1 Basic HITL Setup

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

@dataclass
class RuntimeContext:
    db: SQLDatabase

SYSTEM_PROMPT = """You are a SQLite analyst.
Use execute_sql for queries. Limit to 5 rows.
If the database is offline, ask user to try again later.
"""

# Create agent with HITL middleware
hitl_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=InMemorySaver(),  # Required for HITL
    context_schema=RuntimeContext,
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "execute_sql": {"allowed_decisions": ["approve", "reject"]}
            },
        ),
    ],
)

## 3.2 Handling Interrupts

When the agent wants to use a tool, it will pause and wait for approval:

In [ ]:
from langgraph.types import Command

config = {"configurable": {"thread_id": "hitl-demo-1"}}

# Start a query that requires tool use
result = hitl_agent.invoke(
    {"messages": "What are the names of all employees?"},
    config=config,
    context=RuntimeContext(db=db)
)

# Check if we hit an interrupt
if "__interrupt__" in result:
    interrupt_info = result['__interrupt__'][-1].value['action_requests'][-1]
    print("INTERRUPT DETECTED!")
    print(f"Tool: {interrupt_info['tool_name']}")
    print(f"Args: {interrupt_info['args']}")
    print("\nWaiting for human decision...")
else:
    print(result["messages"][-1].content)

In [ ]:
# REJECT the tool call (simulating database being offline)
result = hitl_agent.invoke(
    Command(
        resume={"decisions": [{"type": "reject", "message": "Database is currently offline."}]}
    ),
    config=config,
    context=RuntimeContext(db=db),
)

print("After rejection:")
print(result["messages"][-1].content)

## 3.3 Approving Tool Calls

In [ ]:
# New conversation with approvals
config = {"configurable": {"thread_id": "hitl-demo-2"}}

result = hitl_agent.invoke(
    {"messages": "What are the names of all employees?"},
    config=config,
    context=RuntimeContext(db=db)
)

# Auto-approve all tool calls until we get a final answer
while "__interrupt__" in result:
    interrupt_info = result['__interrupt__'][-1].value['action_requests'][-1]
    print(f"Approving: {interrupt_info['tool_name']}({interrupt_info['args']})")
    
    result = hitl_agent.invoke(
        Command(resume={"decisions": [{"type": "approve"}]}),
        config=config,
        context=RuntimeContext(db=db),
    )

print("\nFinal Answer:")
print(result["messages"][-1].content)

## 3.4 Interactive Approval Function

In a real application, you'd prompt the user for approval:

In [ ]:
def run_with_approval(agent, messages, config, context):
    """Run agent with interactive approval for tool calls."""
    result = agent.invoke(
        {"messages": messages},
        config=config,
        context=context
    )
    
    while "__interrupt__" in result:
        interrupt_info = result['__interrupt__'][-1].value['action_requests'][-1]
        
        print("\n" + "="*50)
        print("APPROVAL REQUIRED")
        print(f"Tool: {interrupt_info['tool_name']}")
        print(f"Arguments: {interrupt_info['args']}")
        print("="*50)
        
        # In a real app, this would be user input
        # For demo, we'll auto-approve
        decision = "approve"  # or input("Approve? (y/n): ").lower() == 'y'
        
        if decision == "approve":
            print("Approved!")
            result = agent.invoke(
                Command(resume={"decisions": [{"type": "approve"}]}),
                config=config,
                context=context,
            )
        else:
            print("Rejected!")
            result = agent.invoke(
                Command(resume={"decisions": [{"type": "reject", "message": "User denied"}]}),
                config=config,
                context=context,
            )
    
    return result

# Test the interactive function
config = {"configurable": {"thread_id": "hitl-demo-3"}}
result = run_with_approval(
    hitl_agent,
    "How many tracks are in each genre?",
    config,
    RuntimeContext(db=db)
)

print("\nFinal Answer:")
print(result["messages"][-1].content)

---

## Summary

In this notebook, we covered:

1. **Structured Output** - Extracting typed data:
   - `TypedDict` for simple schemas
   - Pydantic `BaseModel` for validation and complex types
   - Nested structures with Lists and Optional fields
   - Access via `result["structured_response"]`

2. **Dynamic Prompts** - Runtime customization:
   - `@dynamic_prompt` decorator
   - Access runtime context via `request.runtime.context`
   - Role-based access control example

3. **Human-in-the-Loop** - Approval workflows:
   - `HumanInTheLoopMiddleware` for tool approval
   - Handle interrupts with `"__interrupt__"` key
   - Resume with `Command(resume=...)` 
   - Approve or reject tool calls

---

**Next:** [Notebook 4: Modern RAG with LangChain](./2.0-modern-rag-langchain.ipynb)